#### 1. Import pandas library

In [2]:
import pandas as pd

#### 2. Load data (users.csv, posts.csv)

In [7]:
users = pd.read_csv('../your-code/users.csv')
posts = pd.read_csv('../your-code/posts.csv')

#### 3. Rename id column to user_id

In [8]:
users.rename(columns = {'id':'user_id'}, inplace = True)

#### 4. Rename id column to post_id and owner_user_id to user_id

In [9]:
posts.rename(columns = {'id':'post_id','owner_user_id':'user_id'}, inplace = True)

#### 5. Define new dataframes for users and posts with the following selected columns:
    **users columns**: user_id, reputation,views,up_votes,down_votes
    **posts columns**: post_id, score,user_id,view_count,comment_count

In [10]:
users = users[['user_id', 'reputation', 'views', 'up_votes', 'down_votes']]
users.head()

,user_id,reputation,views,up_votes,down_votes
0,107658,305,91,2,0
1,218597,6559,374,361,25
2,326360,1802,97,40,14
3,379556,2335,261,125,10
4,450456,7746,3598,256,50


In [11]:
posts = posts[['post_id', 'score', 'user_id', 'view_count', 'comment_count']]
posts.head()

,post_id,score,user_id,view_count,comment_count
0,30336926,1,NaN,14,3
1,36873524,1,NaN,18,0
2,36605876,1,NaN,19,0
3,36718461,1,NaN,17,0
4,30434893,0,NaN,15,0


#### 6. Merge both dataframes, users and posts. 
You will need to make a [merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) of posts and users dataframes.

In [15]:
bothdf = pd.merge(users, posts)

In [16]:
bothdf

,user_id,reputation,views,up_votes,down_votes,post_id,score,view_count,comment_count
0,450456,7746,3598,256,50,34989087,0,19,0
1,450456,7746,3598,256,50,4142174,0,15,0
2,1517244,4172,224,1362,32,37211628,2,18,0
3,1870509,897,105,86,2,30260248,1,11,4
4,101719,4055,332,174,16,34528617,0,18,0
...,...,...,...,...,...,...,...,...,...
1060,4295902,1,4,0,0,31038301,0,10,0
1061,2856201,11,21,0,0,19348144,0,19,2
1062,5882818,11,3,0,0,35199880,0,20,2
1063,6241297,13,2,0,0,36798683,0,12,0


#### 7. How many missing values do you have in your merged dataframe? 

In [18]:
bothdf.isna().sum() #No missing values

user_id          0
reputation       0
views            0
up_votes         0
down_votes       0
post_id          0
score            0
view_count       0
comment_count    0
dtype: int64

#### Bonus: Identify extreme values in your merged dataframe, create a dataframe called outliers with the same columns as our data set and calculate the bounds. The values of the outliers dataframe will be the values of the merged_df that fall outside that bounds. You will need to save your outliers dataframe to a csv file on your-code folder. Hint: post_id cannot have outliers!

In [ ]:
# Did we make this in our lessons??